In [ ]:
#r "nuget: Shioaji, 1.1.8"

In [2]:
using System.IO;
using System.Text.Json;
using Sinopac.Shioaji;
using System.Linq;

Shioaji _api = new Shioaji();
// string jsonString = File.ReadAllText("Sinopac.json");
// JsonElement root = JsonDocument.Parse(jsonString).RootElement;
// _api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());
// _api.ca_activate(@"Sinopac.pfx", root.GetProperty("ca_passwd").GetString(), root.GetProperty("person_id").GetString());

_api.Login("", "");

Connecting as shioaji@sinopac on 203.66.91.161:80...


In [ ]:
var src = _api.ListProfitLossSummary(DateTime.Now.AddDays(-14).ToString("yyyy-MM-dd"), DateTime.Now.ToString("yyyy-MM-dd"), _api.FutureAccount).profitloss_summary;

Dictionary<string, List<object>> retDict = new Dictionary<string, List<object>>();
foreach (var i in src)
{
    List<object> _retDict = new List<object>();
    _retDict.Add("_" + i.code.Substring(3, 7).PadLeft(10));
    _retDict.Add(i.direction.PadRight(10)+'_');
    _retDict.Add(i.quantity);
    _retDict.Add(i.entry_price);
    _retDict.Add(i.cover_price);
    _retDict.Add(i.pnl);
    retDict.Add(i.code, _retDict);
}
retDict.Display();
src

In [ ]:
//        _api.ListPositionDetail(_api.FutureAccount)
var src = _api.ListPositions(_api.FutureAccount);
Dictionary<string, List<object>> retDict = new Dictionary<string, List<object>>();
foreach (var i in src)
{
    List<object> _retDict = new List<object>();
    _retDict.Add(i.code.Substring(3, 7));
    _retDict.Add(i.direction);
    _retDict.Add(i.quantity);
    _retDict.Add(i.price);
    _retDict.Add(i.last_price);
    _retDict.Add(i.pnl);
    retDict.Add(i.code, _retDict);
}
retDict.Display();
src

In [58]:
// _api.Subscribe(_api.Contracts.Stocks["TSE"]["2330"], QuoteType.tick);

_api.Subscribe(
    contract:_api.Contracts.Futures["TXF"]["TXFR1"], 
    quoteType:QuoteType.tick,
    version:QuoteVersion.v1
);


List<Dictionary<string, object>> combinedList = new List<Dictionary<string, object>>();
private void myQuoteCB_v1(Exchange exchange, dynamic tick)
{
    Dictionary<string, object> dict = new Dictionary<string, object>();
    dict.Add(tick.GetType().Name, tick);
    combinedList.Add(dict);
}

_api.SetQuoteCallback_v1(myQuoteCB_v1);

Response Code: 0 | Event code: 16 | Info: TIC/v1/FOP/*/TFE/TXFA4 | Event: SubscriptionOk


In [ ]:
var _contract = _api.Contracts.Futures["MXF"]["MXF202308"];    
var _futOptOrder = new FutOptOrder()
{
    action = "Sell",
    price = 16365,
    quantity = 1, price_type = "LMT", order_type = "ROD"//, octype = OCType.DayTrade
};
var _trade = _api.PlaceOrder(_contract, _futOptOrder);

In [ ]:
// var StopTime = DateTime.Now.AddMinutes(0.5);
var StopTime = new DateTime(2023,8,3,5,0,0);
var positions = _api.ListPositions(_api.FutureAccount);
// while (true) 
while (!string.IsNullOrEmpty(positions[0].code)) 
{
    Decimal _price = 999;  // 這裡Decimal 下面再 Convert.ToDouble(_price) 是可下單的；試別種
    

    if (positions[0].pnl >= 50 )
    {
        // ===========行情價=============
        _price = positions[0].last_price + 1;
    }
    else if (positions[0].pnl < -10000)
    {
        // =========成本價==========
        _price = positions[0].price;
    }


    if (_price != 999)
    {
        var sym1 =  _api.ListPositions(account:_api.FutureAccount)[0].code.ToString().Substring(0,3);  // TX2
        var sym2 =  _api.ListPositions(account:_api.FutureAccount)[0].code.ToString().Substring(3,5);  // 16850
        // var sym3 = (P ? C : P)  // 畢竟下面"P"目前寫死
        var _contract = _api.Contracts.Options[sym1][sym1 + "202308" + sym2 + "C"];                    // opt_code["TX220230816850C"];
        var _futOptOrder = new FutOptOrder()
        {
            action = "Sell",  // 只會把買權平倉，應該不必考慮先賣後買 action = (positions[0].direction == "Buy") ? "Sell" : positions[0].direction,
            price = Convert.ToDouble(_price),
            quantity = 1, price_type = "LMT", order_type = "ROD", octype = "Cover"
        };
        var _trade = _api.PlaceOrder(_contract, _futOptOrder);
        break;
    }
    else if (DateTime.Now >= StopTime)
    {
        break;
    }
    else
    {
        Thread.Sleep(10_000);
    }
}

In [ ]:
/*
小單平倉每X秒移價
*/
var StopTime = DateTime.Now.AddMinutes(0.5);   // var StopTime = new DateTime(2023,7,27,5,0,0);
while (true) 
{
    var positions = _api.ListPositions(account:_api.FutureAccount);
    Decimal _price = 0;  // 這裡Decimal 下面再 Convert.ToDouble(_price) 是可下單的；試別種
    if (positions[0].code == "MXF" && positions[0].pnl > 100)
    {
        // ===========行情價=============
        _price = positions[0].last_price;
    }
    else if (positions[0].code == "MXF" && positions[0].pnl < -1000)
    {
        // =========成本價==========
        _price = positions[0].price;
    }
    var _contract = _api.Contracts.Futures["MXF"]["MXF202308"];    
    var _futOptOrder = new FutOptOrder()
    {
        action = (positions[0].direction == "Buy") ? "Sell" : positions[0].direction,
        price = Convert.ToDouble(_price),
        quantity = 1, price_type = "LMT", order_type = "ROD", octype = OCType.DayTrade
    };
    var _trade = _api.PlaceOrder(_contract, _futOptOrder);


    Thread.Sleep(10_000);
    if (DateTime.Now >= StopTime) // (DateTime.Now >= DateTime(2023,7,15,5,0,0))
    {
        break;
    }
}

In [ ]:
// var StopTime = DateTime.Now.AddMinutes(0.5);
var StopTime = new DateTime(2023,8,3,5,0,0);
var positions = _api.ListPositions(_api.FutureAccount);
// while (true) 
while (!string.IsNullOrEmpty(positions[0].code)) 
{
    Decimal _price = 999;  // 這裡Decimal 下面再 Convert.ToDouble(_price) 是可下單的；試別種
    

    if (positions[0].pnl >= 50 )
    {
        // ===========行情價=============
        _price = positions[0].last_price + 1;
    }
    else if (positions[0].pnl < -10000)
    {
        // =========成本價==========
        _price = positions[0].price;
    }


    if (_price != 999)
    {
        var sym1 =  _api.ListPositions(account:_api.FutureAccount)[0].code.ToString().Substring(0,3);  // TX2
        var sym2 =  _api.ListPositions(account:_api.FutureAccount)[0].code.ToString().Substring(3,5);  // 16850
        // var sym3 = (P ? C : P)  // 畢竟下面"P"目前寫死
        var _contract = _api.Contracts.Options[sym1][sym1 + "202308" + sym2 + "C"];                    // _api.Contracts.Options["TX2"]["TX220230816850C"];
        var _futOptOrder = new FutOptOrder()
        {
            action = "Sell",  // 只會把買權平倉，應該不必考慮先賣後買 action = (positions[0].direction == "Buy") ? "Sell" : positions[0].direction,
            price = Convert.ToDouble(_price),
            quantity = 1, price_type = "LMT", order_type = "ROD", octype = "Cover"
        };
        var _trade = _api.PlaceOrder(_contract, _futOptOrder);
        break;
    }
    else if (DateTime.Now >= StopTime)
    {
        break;
    }
    else
    {
        Thread.Sleep(10_000);
    }
}

In [ ]:
/*
小單平倉每X秒移價
*/
var StopTime = DateTime.Now.AddMinutes(0.5);   // var StopTime = new DateTime(2023,7,27,5,0,0);
while (true) 
{
    var positions = _api.ListPositions(account:_api.FutureAccount);
    Decimal _price = 0;  // 這裡Decimal 下面再 Convert.ToDouble(_price) 是可下單的；試別種
    if (positions[0].code == "MXF" && positions[0].pnl > 100)
    {
        // ===========行情價=============
        _price = positions[0].last_price;
    }
    else if (positions[0].code == "MXF" && positions[0].pnl < -1000)
    {
        // =========成本價==========
        _price = positions[0].price;
    }
    var _contract = _api.Contracts.Futures["MXF"]["MXF202308"];    
    var _futOptOrder = new FutOptOrder()
    {
        action = (positions[0].direction == "Buy") ? "Sell" : positions[0].direction,
        price = Convert.ToDouble(_price),
        quantity = 1, price_type = "LMT", order_type = "ROD", octype = OCType.DayTrade
    };
    var _trade = _api.PlaceOrder(_contract, _futOptOrder);


    Thread.Sleep(10_000);
    if (DateTime.Now >= StopTime) // (DateTime.Now >= DateTime(2023,7,15,5,0,0))
    {
        break;
    }
}